In [17]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy import sparse
from scipy.sparse.linalg import norm
from sklearn.model_selection import train_test_split
from typing import Tuple, Mapping, Any

In [19]:
df = pd.read_csv("res\\rating.csv")

In [20]:
df['rating'].describe()

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [21]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [22]:
def split_by_target_column_value(df: pd.DataFrame, test_size: float, target_column: str, target_value: Any,
                                 random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Разделяет DataFrame на обучающий и тестовый наборы на основе определенного значения целевого столбца.
    Выдает в качестве тестового набора test_size от набора, в котором значение в target_column == target_value
    В тренировочном все остальное
    Параметры:
        df (pd.DataFrame): DataFrame для разделения.
        test_size: Пропорция данных, которые следует включить в тестовый набор.
        target_column (str): Название целевого столбца для разделения.
        target_value (Any): Значение целевого столбца, используемое для разделения.
        random_state (int): Зерно для генератора случайных чисел.

    Возвращает:
        Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]: Данные для обучения; тестовые данные для целевой колонки, которые не попали в data,
        тренировочные данные для целевой колоники, которые попали в data
    """
    grouped = df.groupby(by=target_column)
    train_df, test_df = train_test_split(grouped.get_group(target_value), test_size=test_size,
                                         random_state=random_state)
    data = pd.concat([group for ind, group in grouped if ind != target_value], ignore_index=True)
    data = pd.concat([train_df, data], ignore_index=True)
    return data, test_df, train_df


In [23]:
data, test1, train1 = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

In [24]:
data.head()

,userId,movieId,rating,timestamp
0,1,2288,4.0,2004-09-10 03:14:37
1,1,2648,3.5,2004-09-10 03:13:37
2,1,1222,3.5,2005-04-02 23:30:37
3,1,3037,3.5,2005-04-02 23:46:03
4,1,2683,3.5,2004-09-10 03:07:30


In [25]:
test1.head()

,userId,movieId,rating,timestamp
156,1,7045,3.5,2005-04-02 23:50:34
145,1,6242,3.5,2004-09-10 03:13:24
101,1,3081,4.0,2004-09-10 03:15:43
127,1,4911,4.0,2005-04-02 23:52:41
141,1,5898,3.5,2005-04-02 23:53:22


In [26]:
train1.head()

,userId,movieId,rating,timestamp
79,1,2288,4.0,2004-09-10 03:14:37
84,1,2648,3.5,2004-09-10 03:13:37
39,1,1222,3.5,2005-04-02 23:30:37
100,1,3037,3.5,2005-04-02 23:46:03
86,1,2683,3.5,2004-09-10 03:07:30


In [54]:
from abc import ABC, abstractmethod
from typing import Mapping, Tuple, Any, Literal
MappingIntToObject = Mapping[int, Tuple[Any, Literal['source', 'target']]]
from sklearn.preprocessing import normalize
        
# https://github.com/danieljunhee/Tutorial-on-Personalized-PageRank/blob/master/Personalized_PageRank_Tutorial.ipynb
class PageRank():
    
    def __init__(self, a: float = 0.8, b: float = 0.15, c: float=0.05) -> None:
        # Decide values of a,b,c. For this tutorial, let's use a = 0.8, b = 0.15, c = 0.05.
        self.a = a
        self.b = b
        self.c = c
        
    def __create_transition_matrix_from_pd(self, data: pd.DataFrame,
               source: str, target: str, edge_attr: str,
               personalization: Any) -> Tuple[sparse.csr_matrix, MappingIntToObject]:
    
        obj_source_to_N = {}
        obj_target_to_N = {}
        N_to_obj = {}
        
        source_uniq = set(data[source])
        for index, obj in zip(range(len(source_uniq)), source_uniq):
            obj_source_to_N[obj] = index
            N_to_obj[index] = (obj, 'source')
            
        
        target_uniq = set(data[target])
        for index, obj in zip(range(len(source_uniq), len(source_uniq) + len(target_uniq)), target_uniq):
            obj_target_to_N[obj] = index
            N_to_obj[index] = (obj, 'target')
        
        count_top = len(N_to_obj)
        transition_matrix = sparse.lil_matrix((count_top, count_top))
        
        # Функция для отображения значений согласно словарю
        def map_source_to_N(value):
            return obj_source_to_N[value]

        def map_target_to_N(value):
            return obj_target_to_N[value]

        # Создаем векторизованную версию функций
        vectorized_source_to_N = np.vectorize(map_source_to_N)
        vectorized_target_to_N = np.vectorize(map_target_to_N)

        # Применяем векторизованные функции к массивам
        start_nodes = vectorized_source_to_N(data[source].to_numpy())  # Начальные вершины ребер
        end_nodes = vectorized_target_to_N(data[target].to_numpy())    # Конечные вершины ребер
        weights = np.array(data[edge_attr])
        
        # Заполнение матрицы значениями из массивов
        transition_matrix[start_nodes, end_nodes] = weights

        # Если граф неориентированный, добавьте обратные ребра
        transition_matrix[end_nodes, start_nodes] = weights
        
        # Получаем суммы элементов каждой строки в виде numpy массива
        
        transition_matrix_csr = sparse.csr_matrix(transition_matrix)
        del transition_matrix
        
        row_sums = transition_matrix_csr.sum(axis=1)
        transition_matrix_csr /= row_sums
        
        return (transition_matrix_csr,
                transition_matrix_csr.getrow(obj_source_to_N[personalization]).transpose(), N_to_obj)
    
    def get_pagerank(self, data: pd.DataFrame, source: str, target: str,
                     edge_attr: str, personalize_by: Any) -> pd.DataFrame:
        
        (transition_matrix, user_person_vec,
         N_to_obj) = self.__create_transition_matrix_from_pd(data, source,
                                                             target, edge_attr, personalize_by)
         
        pagerank = self.__get_pagerank(transition_matrix, user_person_vec)
         
        def map_source(x):
            return N_to_obj[x][0] if N_to_obj[x][1] == 'target' else 'x' # Если элемента нет в словаре, вернуть исходное значение
        # Применение словаря к каждому элементу массива
        vfunc = np.vectorize(map_source)
        result = vfunc(np.arange(len(pagerank)))
        empty_indices = np.where(result == 'x')[0]
        source_column = result[~empty_indices]
        pagerank_column = pagerank[~empty_indices]
    
        return pd.DataFrame({'target': source_column , 'pagerank': pagerank_column})
    
    def __get_pagerank(self, transition_matrix: sparse.csr_matrix,
                       user_person_vec: sparse.csr_matrix, max_iter: int = 200) -> np.ndarray:
        
        # Initialize PageRank vector: height x 1 vector of all 1/height 's
        # We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
        # is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
        
        height = transition_matrix.shape[0]
        user_pagerank_vec = sparse.csr_matrix(np.ones((height,1)) / height)
        transition_matrix *= self.a
        user_person_vec *= self.b
        sup_vec = self.c*sparse.csr_matrix((1/height)*np.ones((height,1)))

        # Update the PageRank vector until convergence! Our convergence criterion is to see whether
        # the magnitude of the difference of the PageRank vector after an update is 0
        converged = False
        count_iter = 0
        while (not converged):
            count_iter += 1
            # (deep) copy the current PageRank vector to compare with the updated vector
            old_user_pagerank_vec = user_pagerank_vec.copy()
            
            # Update!
            user_pagerank_vec = transition_matrix.dot(user_pagerank_vec) \
                                + user_person_vec.multiply(user_pagerank_vec) \
                                + sup_vec 
            user_pagerank_vec /= user_pagerank_vec.sum()
            
            # Compare with the PageRank vector before the update. Check the magnitude of the difference of the vector
            print("magnitude of difference vector:", norm(user_pagerank_vec-old_user_pagerank_vec))
            
            if norm(user_pagerank_vec-old_user_pagerank_vec) == 0 or count_iter == max_iter:
                converged = True
        return np.concatenate(user_pagerank_vec.toarray())
    
  
data_test = pd.DataFrame({
    'source': ['A', 'A', 'B', 'B', 'C'],
    'target': ['X', 'Y', 'X', 'Z', 'Z'],
    'edge_attr': [4, 5, 4, 5, 3]
})  
    
pg = PageRank()
pg.get_pagerank(data_test, 'source', 'target', 'edge_attr', 'A')

magnitude of difference vector: 0.01664775949476432
magnitude of difference vector: 0.01169531326139496
magnitude of difference vector: 0.01098154774709629
magnitude of difference vector: 0.007684189221429654
magnitude of difference vector: 0.00763308665100461
magnitude of difference vector: 0.005560144897079669
magnitude of difference vector: 0.005576577577363453
magnitude of difference vector: 0.004253977985326386
magnitude of difference vector: 0.0042152795172867834
magnitude of difference vector: 0.003347546429944893
magnitude of difference vector: 0.0032523473913362204
magnitude of difference vector: 0.002665082360763458
magnitude of difference vector: 0.0025392850444841174
magnitude of difference vector: 0.0021297184441994104
magnitude of difference vector: 0.0019961465405946553
magnitude of difference vector: 0.0017027419414969258
magnitude of difference vector: 0.0015755223716870197
magnitude of difference vector: 0.0013604820038521652
magnitude of difference vector: 0.00124660

,target,pagerank
0,Z,0.149920
1,Y,0.190878
2,X,0.176633


In [55]:
pg = PageRank()
rank = pg.get_pagerank(data, 'userId', 'movieId', 'rating', 1)


magnitude of difference vector: 1.1509953870291128e-07
magnitude of difference vector: 2.1082979705417937e-07
magnitude of difference vector: 6.32110274592432e-08
magnitude of difference vector: 8.917281538009349e-08
magnitude of difference vector: 7.406594114859456e-08
magnitude of difference vector: 7.643182007152574e-08
magnitude of difference vector: 7.045959353827726e-08
magnitude of difference vector: 6.75338628332391e-08
magnitude of difference vector: 6.337590392018457e-08
magnitude of difference vector: 5.981158704181234e-08
magnitude of difference vector: 5.629066892588132e-08
magnitude of difference vector: 5.297824158065424e-08
magnitude of difference vector: 4.987105635579513e-08
magnitude of difference vector: 4.692227214902354e-08
magnitude of difference vector: 4.416513337125414e-08
magnitude of difference vector: 4.155648933796223e-08
magnitude of difference vector: 3.9110922199689076e-08
magnitude of difference vector: 3.680339783058147e-08
magnitude of difference vec

In [56]:
print(rank.sort_values(by='pagerank', ascending=True).head())


       target  pagerank
12611   70684  0.000006
13970   59895  0.000006
11224   78873  0.000006
2244   116704  0.000006
3086   112909  0.000006


In [60]:
rank.value_counts()

target  pagerank
1       0.000006    1
x       0.000006    1
        0.000006    1
        0.000006    1
        0.000006    1
                   ..
        0.000006    1
        0.000006    1
        0.000006    1
        0.000006    1
        0.000006    1
Name: count, Length: 138493, dtype: int64

In [57]:
rank['pagerank'].sum()

0.8381449625129656